# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [1]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t

In [2]:
# rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
rc = ut.read_config("../config/test.toml")
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [3]:
import vipy.simulation.scan as scan

In [4]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [15]:
time = ut.calc_time_steps(rc)
print(time)

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:20:00.000'
 '2016:095:00:20:10.000' '2016:095:00:40:00.000' '2016:095:00:40:10.000'
 '2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:20:00.000'
 '2016:095:01:20:10.000' '2016:095:01:20:20.000']


## Calculate Baselines for one measurement

In [24]:
baselines = scan.get_baselines(src_crd, time[:], array_layout)
len(baselines.u[baselines.valid == True])

86

In [57]:
print(len(baselines.valid))
valid = baselines.valid.reshape(-1, 28)
mask = np.array([(np.array(valid[i]).astype(bool)) & (np.array(valid[i+1]).astype(bool)) for i in range(valid.shape[0] - 1)])
mask.shape

308


(10, 28)

In [64]:
bas_st1 = baselines.st1.reshape(-1, 28)
bas_st2 = baselines.st2.reshape(-1, 28)
bas_st1.shape

(11, 28)

In [78]:
st1_start = bas_st1[:-1][mask]
st1_stop = bas_st1[1:][mask]
print(st1_stop.shape == st1_start.shape)

st2_start = bas_st2[:-1][mask]
st2_stop = bas_st2[1:][mask]
print(st2_stop.shape == st2_start.shape)

print(st1_start.shape == st2_start.shape)

True
True
True


In [80]:
st2_start

array([Station(st_num=5, name='PdBI', x=4523998.4, y=468045.24, z=4460309.76, diam=7.0, el_low=15, el_high=85, sefd=1600, altitude=2550),
       Station(st_num=7, name='GLT', x=1500692.0, y=-1191735.0, z=6066409.0, diam=12.0, el_low=15, el_high=85, sefd=4744, altitude=3210),
       Station(st_num=5, name='PdBI', x=4523998.4, y=468045.24, z=4460309.76, diam=7.0, el_low=15, el_high=85, sefd=1600, altitude=2550),
       Station(st_num=4, name='PV', x=5088967.9, y=-301681.6, z=3825015.8, diam=30.0, el_low=15, el_high=85, sefd=2900, altitude=2850),
       Station(st_num=7, name='GLT', x=1500692.0, y=-1191735.0, z=6066409.0, diam=12.0, el_low=15, el_high=85, sefd=4744, altitude=3210),
       Station(st_num=7, name='GLT', x=1500692.0, y=-1191735.0, z=6066409.0, diam=12.0, el_low=15, el_high=85, sefd=4744, altitude=3210),
       Station(st_num=5, name='PdBI', x=4523998.4, y=468045.24, z=4460309.76, diam=7.0, el_low=15, el_high=85, sefd=1600, altitude=2550),
       Station(st_num=7, name='GLT',

## Calculate Mueller/Jones matrices

In [7]:
import numpy as np
import torch

In [8]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

/home/kschmidt_local/.local/anaconda3/envs/vipy/lib/python3.9/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/kschmidt_local/radio_nets/vipy/vipy/simulation/scan.py:319: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  K = torch.tensor(getK(baselines, lm, wave))


CPU times: user 9.49 s, sys: 3.04 s, total: 12.5 s
Wall time: 2.18 s


## Open Source Image and compute Stokes vector

In [9]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [10]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [11]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
print(delta_l)

integral = scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

0.0001830982550547064
6.770906209945679
CPU times: user 3.12 s, sys: 1.16 s, total: 4.29 s
Wall time: 622 ms


In [12]:
integral.shape

torch.Size([84, 4])

## All scan loop

In [13]:
from tqdm import tqdm

In [14]:
torch.set_num_threads(12)

In [15]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [16]:
integral2 = np.array([])
for i in tqdm(range(5)):
    t = time[i*2:(i+1)*2+1]
    print(t)
    baselines = scan.get_baselines(src_crd, t, array_layout)
    print(len(baselines.u[baselines.valid==True]))
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)
    print(JJ_f1.shape)

    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
    delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
    print(delta_l)

    integral2 = np.append(integral2,scan.integrate(JJ_f1, JJ_f2, I, 1, delta_t, delta_f, delta_l, delta_m))
    print(scan.integrate(JJ_f1, JJ_f2, I, 1, delta_t, delta_f, delta_l, delta_m).shape)


  0%|          | 0/5 [00:00<?, ?it/s]

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:20:00.000']
18


/home/kschmidt_local/.local/anaconda3/envs/vipy/lib/python3.9/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/kschmidt_local/radio_nets/vipy/vipy/simulation/scan.py:319: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  K = torch.tensor(getK(baselines, lm, wave))


torch.Size([256, 256, 18, 4, 4])
0.0001830982550547064


 20%|██        | 1/5 [00:00<00:03,  1.19it/s]

...

torch.Size([21, 4])
['2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:20:00.000']
30
torch.Size([256, 256, 30, 4, 4])
0.0001830982550547064


 80%|████████  | 4/5 [00:03<00:01,  1.02s/it]

torch.Size([29, 4])
['2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000']
30
torch.Size([256, 256, 30, 4, 4])
0.0001830982550547064


100%|██████████| 5/5 [00:05<00:00,  1.01s/it]

torch.Size([29, 4])


In [17]:
integral2 = integral2.reshape(-1, 4)
integral2.shape

(113, 4)

In [18]:
I

tensor([[[4.2461e-07+0.j, 3.0025e-07+0.j, 3.0025e-07+0.j, 0.0000e+00+0.j],
         [2.1231e-07+0.j, 1.5012e-07+0.j, 1.5012e-07+0.j, 0.0000e+00+0.j],
         [6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         ...,
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j],
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j],
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j]],

        [[6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         [6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         [8.4922e-07+0.j, 6.0049e-07+0.j, 6.0049e-07+0.j, 0.0000e+00+0.j],
         ...,
         [8.4922e-07+0.j, 6.0049e-07+0.j, 6.0049e-07+0.j, 0.0000e+00+0.j],
         [3.1846e-06+0.j, 2.2518e-06+0.j, 2.2518e-06+0.j, 0.0000e+00+0.j],
         [8.4922e-06+0.j, 6.0049e-06+0.j, 6.0049e-06+0.j, 0.0000e+00+0.j]],

        [[4.2461e-07+0.j, 3.0025e-07+0.j, 3.0025e-07+0.j, 0.0000e+00